In [1]:
# Basic Neccesities 
import pandas as pd
import string
import re


#NLTK Packages
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet
from nltk.corpus import stopwords
import wordninja

In [2]:
data = pd.read_csv('data/train.txt')
pd.options.display.max_colwidth = 500
data.head()

,tweet_id,sentiment,tweet_text
0,264183816548130816,positive,Gas by my house hit $3.39!!!! I\u2019m going to Chapel Hill on Sat. :)
1,263405084770172928,negative,Theo Walcott is still shit\u002c watch Rafa and Johnny deal with him on Saturday.
2,262163168678248449,negative,its not that I\u2019m a GSP fan\u002c i just hate Nick Diaz. can\u2019t wait for february.
3,264249301910310912,negative,Iranian general says Israel\u2019s Iron Dome can\u2019t deal with their missiles (keep talking like that and we may end up finding out)
4,262682041215234048,neutral,Tehran\u002c Mon Amour: Obama Tried to Establish Ties with the Mullahs http://t.co/TZZzrrKa via @PJMedia_com No Barack Obama - Vote Mitt Romney


In [3]:
text = list(data['tweet_text'])

In [4]:
# Emoji's Dictionary
emojis = {':-)':'happy',':)':'happy',':D':'happy',':o)':'happy',':]':'happy',':3':'happy',':c)':'happy', ':>':'happy', 
          '=]':'happy','8)':'happy',';)':'happy',';-)':'happy',':d':'happy',';d':'happy','xx':'happy','.xx':'happy',
          '<33':'happy','<3':'happy','>.<':'happy','|;-)':'happy',
          
          '>:[':'sad',':-(':'sad',':(':'sad',':-c':'sad',':c':'sad',':-<':'sad',':<':'sad', ':-[':'sad',':[':'sad', 
          ':{':'sad',':’-(':'sad',':’(':'sad','>:':'sad','>:/':'sad',':-/':'sad',':-.':'sad',':- :':'sad','=/=':'sad',
          ':L':'sad','=L':'sad',':S':'sad',
          
          ':-||':'angry',':@':'angry','>:(':'angry',
    
         'D:<':'shocked','D:':'shocked','D8':'shocked','D;':'shocked','D=':'shocked','DX':'shocked','v.v':'shocked',
          'D-’':'shocked',';o':'shocked',';O':'shocked','XDD':'shocked','XD':'shocked','>:O':'shocked',':-O':'shocked',
          ':O':'shocked',':-o':'shocked',':o':'shocked','8O':'shocked','O_O':'shocked','o-o':'shocked','O_o':'shocked',
          'o_O':'shocked','o_o':'shocked','O-O':'shocked',
          
          ':|':'neutral',':-|':'netural',
          
         }


# Contradictions and Abbreviations Dictionary

replace = {"dont":"do not","don't":"do not",
              "aren't":"are not","arent":"are not","cant":"can not","can't":"can not",
              "couldn't":"could not",'couldn':"could not","couldnt":"could not",
              'didn':"did not","didn't":"did not","didnt":"did not",
              'doesn':"does not","doesn't":"does not","doesnt":"does not",
              'hadn':"had not","hadn't":"had not","hadnt":"had not",
              'hasn':"has not", "hasn't":"has not", 'haven':"have not",
              "haven't":"have not","haven":"have not","havent":"have not",
              'isn':"is not", "isn't":"is not","isnt":"is not",
              'mightn':"might not","mightn't":"might not","mightnt":"might not",
              'mustn':"must not", "mustn't":"must not","mustnt":"must not",
              'needn':"need not","needn't":"need not","neednt":"need not",
              'shan':"shall not", "shan't":"shall not","shant":"shall not",
              'shouldn':"should not", "shouldn't":"should not","shouldnt":"should not",
              'wasn':"was not","wasn't":"was not","wasnt":"was not",
              'weren':"were not", "weren't":"were not","werent":"were not",
              'won':'will not', "won't":"will not","wont":"will not",
              'wouldn':"would not","wouldn't":"would not","wouldnt":"would not",
              "aint":"am not","ain't":"am not",'im':'i am',"i'm":'i am',
              'yall':'you all',"youve":'you have',"you've":'you have',"i'll":'i will','ill':'i will','u':'you',
              'amp':'is not my problem','smh':'shaking my head','lol':'laughing out loud',
              'jan':'january','feb':'february','mar':'march','apr':'april','jun':'june','aug':'august',
              'nov':'november','sept':'september','oct':'october','dec':'december',
              'mon':'monday','tue':'tuesday','tues':'tuesday','wed':'wednesday','thur':'thursday','fri':'friday',
              '@':'at','tis':'this','ohlawd':'oh lord','yas':'yes','yass':'yes','hmm':'',
              'tho':'though','thov':'though','btw':'by the way',"cmon":'come on',"c'mon":'come on','cuz':'cause',
              'diff':'difference','sayin':'saying','drivin':'driving',
              'lmfao':'laughing my fucking ass off','lmao':'laughing my ass off','rofl':'rolling on the floor laughing',
              'lmaoo':'laughing my ass off'}


# Stop Words 

stop_words = stopwords.words('english')
stop_words.remove('no')
stop_words.remove('not')
stop_words.append('tomorrow')
stop_words.append('today')
stop_words.extend(['monday','tuesday','wednesday','thursday','friday','saturday','sunday','january','february','march','april','may','june','july','august','september','october','november','december'])


# Lemmatization with Parts of Speech Tagging
lemmatizer = WordNetLemmatizer()

# Parts of Speech Tagging
def nltk_tag_to_wordnet_tag(nltk_tag):
    if nltk_tag.startswith('J'):
        return wordnet.ADJ
    elif nltk_tag.startswith('V'):
        return wordnet.VERB
    elif nltk_tag.startswith('N'):
        return wordnet.NOUN
    elif nltk_tag.startswith('R'):
        return wordnet.ADV
    else:          
        return None

def lemmatize_sentence(sentence):
    #tokenize the sentence and find the POS tag for each token
    nltk_tagged = nltk.pos_tag(nltk.word_tokenize(sentence))  
    #tuple of (token, wordnet_tag)
    wordnet_tagged = map(lambda x: (x[0], nltk_tag_to_wordnet_tag(x[1])), nltk_tagged)
    lemmatized_sentence = []
    for word, tag in wordnet_tagged:
        if tag is None:
            #if there is no available tag, append the token as is
            lemmatized_sentence.append(word)
        else:        
            #else use the tag to lemmatize the token
            lemmatized_sentence.append(lemmatizer.lemmatize(word, tag))
    return " ".join(lemmatized_sentence)

In [5]:
# Data Cleaning Function
def cleaning(text):
    clean=[]
    i=0
    for tweet in text:
        i+=1
        if i%5000==0:
            print('Completed:',i)
        if r'\u2019' in tweet: 
            tweet = tweet.replace('\\u2019',"'")
        if r'\u002c' in tweet:
            tweet= tweet.replace('\\u002c',',') 
        if r'\""' in tweet:
            tweet= tweet.replace('\""','"') 
        if '&amp;' in tweet:
            tweet= tweet.replace('&amp;','&')

        
        tweet = re.sub(r"((http://)[^ ]*|(https://)[^ ]*|( www\.)[^ ]*|(http)|(http:/))",'',tweet)
        
        tweet = re.sub('RT','',tweet)
        
        tweet = re.sub('@[A-Za-z0-9_]+','', tweet) 
        
        tweet = tweet.lower()
        
        tweet = re.sub(r"(.)\1\1+", r"\1\1", tweet)
        
        tweet = re.sub(r"(\d+th)|(\d+rd)|(\d+st)|(\d+nd)|(\d+)",'',tweet)
        
        emojis_list = emojis.keys()
        for emoji in emojis_list:
            if emoji in tweet:
                tweet=tweet.replace(emoji,' '+ emojis[emoji])
            
        tweet = tweet.translate(str.maketrans('', '', string.punctuation))
              
        tweet_text =''
        for word in tweet.split():
            if len(word)>6:
                word = wordninja.split(word)
                word = ' '.join(word)
            tweet_text += (word+' ')
            
        tweet = lemmatize_sentence(tweet_text)
        
        tweet_text =''
        for word in tweet.split():
            if word in replace:
                word = replace[word]
            for wd in word.split():
                if wd not in stop_words:
                    tweet_text += (word+' ')
        tweet = tweet_text
        clean.append(tweet)
    return clean

In [6]:
clean_text = cleaning(text)

Completed: 5000
Completed: 10000
Completed: 15000
Completed: 20000


In [7]:
data['clean_text']=clean_text
data.head()

,tweet_id,sentiment,tweet_text,clean_text
0,264183816548130816,positive,Gas by my house hit $3.39!!!! I\u2019m going to Chapel Hill on Sat. :),gas house hit go chapel hill sat happy
1,263405084770172928,negative,Theo Walcott is still shit\u002c watch Rafa and Johnny deal with him on Saturday.,theo walcott still shit watch rafa johnny deal
2,262163168678248449,negative,its not that I\u2019m a GSP fan\u002c i just hate Nick Diaz. can\u2019t wait for february.,not gsp fan hate nick diaz can not wait
3,264249301910310912,negative,Iranian general says Israel\u2019s Iron Dome can\u2019t deal with their missiles (keep talking like that and we may end up finding out),iranian general say israel iron dome can not deal missile keep talk like end find
4,262682041215234048,neutral,Tehran\u002c Mon Amour: Obama Tried to Establish Ties with the Mullahs http://t.co/TZZzrrKa via @PJMedia_com No Barack Obama - Vote Mitt Romney,tehran amour obama try establish tie mullah via no barack obama vote mitt romney


In [8]:
# Saving Clean Dataset

data.to_csv('data/train_clean_data.csv')

In [9]:
# Test data - Cleaning Process:

test = pd.read_csv('data/test_samples.txt')
test_tweets = list(test['tweet_text'])
test_clean_text = cleaning(test_tweets)
test['clean_text']= test_clean_text
test.to_csv('data/test_clean_data.csv')
test.head()

Completed: 5000


,tweet_id,tweet_text,clean_text
0,264238274963451904,"@jjuueellzz down in the Atlantic city, ventnor, margate, ocean city area. I'm just waiting for the coordinator to hopefully call me tomorrow",atlantic city ventnor margate ocean city area wait coordinator hopefully call
1,218775148495515649,"Musical awareness: Great Big Beautiful Tomorrow has an ending, Now is the time does not",musical awareness great big beautiful end time not
2,258965201766998017,On Radio786 100.4fm 7:10 Fri Oct 19 Labour analyst Shawn Hattingh: Cosatu's role in the context of unrest in the mining http://t.co/46pjzzl6,radio fm labour analyst shawn hat h cosatu role context unrest mining
3,262926411352903682,"Kapan sih lo ngebuktiin,jan ngomong doang Susah Susah.usaha Aja blm udh nyerah,inget.if you never try you'll never know.cowok kok gentle bgt",kapan sih lo nge buk ti ngo mong doang susah susa hu saha aja blm udh nye rah get never try youll never know co wok kok gentle bgt
4,171874368908050432,"Excuse the connectivity of this live stream, from Baba Amr, so many activists using only one Sat Modem. LIVE http://t.co/U283IhZ5 #Homs",excuse connectivity live stream baba amr many activist use one sit modem live homs
